## Environment setup

In [ ]:
!pip install torch
!pip install -qq transformers
!pip install NRCLex

In [ ]:
import os, sys, re
import pandas as pd

sys.path.insert(0, '/evelynkyl/scifi-sentiment-emotion-classifiction/scripts')

# import functions
from utils import *
from data_processing import *
from character_frequency import *
from sentiment_analysis_funcs import *
from emotion_classification_funcs import *
from eval_funcs import *

## Read the files

In [ ]:
datapath = "/evelynkyl/scifi-sentiment-emotion-classifiction/data"
dystro_str = read_file_str(datapath)
book_names = get_names(datapath)

## Preprocessing

In [ ]:
# Clean the texts, including replacing negative contraction ('t) to its full form (not), and normalize punctuations
preprocessed_dystro = mass_text_preprocessing(dystro_str)
print('Original: ', dystro_str[1][:1000])
print(" --------------------------------------- ")
print('Processed: ', preprocessed_dystro[1][:1000])

Original:  Everything was perfectly swell.  There were no prisons, no slums, no insane asylums, no cripples, no poverty, no wars.  All diseases were conquered. So was old age.  Death, barring accidents, was an adventure for volunteers.  The population of the United States was stabilized at forty-million souls.  One bright morning in the Chicago Lying-in Hospital, a man named Edward K. Wehling, Jr.,  waited for his wife to give birth. He was the only man waiting.  Not many people were born a day any more.  Wehling was fifty-six, a mere stripling in a population whose average age was one hundred and twenty-nine.  X-rays had revealed that his wife was going to have triplets. The children would be his first.  Young Wehling was hunched in his chair, his head in his hand. He was so rumpled, so still and colorless as to be virtually invisible. His camouflage was perfect, since the waiting room had a disorderly and demoralized air, too. Chairs and ashtrays had been moved away from the walls.  

In [ ]:
# Segement the texts in to list of strings
book_sents = mass_sent_tokenizer(preprocessed_dystro)
# Remove empty lines
book_sents = remove_dot_line(book_sents)
book_sents[0][:10]

[' If it was good enough for your grandfather, forget it .',
 'it is much too good for anyone else!',
 'Gramps Ford, his chin resting on his hands, his hands on the crook of his cane, was staring irascibly at the five-foot television screen that dominated the room.',
 "On the screen, a news commentator was summarizing the day's happenings.",
 'Every thirty seconds or so, Gramps would jab the floor with his cane-tip and shout, "Hell, we did that a hundred years ago!"',
 "Emerald and Lou, coming in from the balcony, where they had been seeking that 2185 A. D. rarity--privacy--were obliged to take seats in the back row, behind Lou's father and mother, brother and sister-in-law, son and daughter-in-law, grandson and wife, granddaughter and husband, great-grandson and wife, nephew and wife, grandnephew and wife, great-grandniece and husband, great-grandnephew and wife--and, of course, Gramps, who was in front of everybody.",
 'All save Gramps, who was somewhat withered and bent, seemed, by 

In [ ]:
# Convert the list of sentences into dataframe for easier data handling
df = pd.DataFrame(book_sents)
df = df.T # transpose  dataframe
df.columns = book_names #rename columns
df = df.replace(np.nan, '')
# combine all books into one column
df_ori = df.melt(value_name='all')['all']
df_ori = pd.DataFrame(df_ori)
df_ori

,all
0,"If it was good enough for your grandfather, f..."
1,it is much too good for anyone else!
2,"Gramps Ford, his chin resting on his hands, hi..."
3,"On the screen, a news commentator was summariz..."
4,"Every thirty seconds or so, Gramps would jab t..."
...,...
59895,The magnitude of the task may be understood wh...
59896,This is the end of the Everhard Manuscript.
59897,It breaks off abruptly in the middle of a...
59898,She must have received warning of the com...


## Character Frequency

1. Find name entities
2. Get the top 3 characters by occurrence count
2. Get each of their dialogues

In [ ]:
print(list_name_entities(book_sents[1]))

['Edward K. Wehling', 'Young Wehling', 'Hitz', 'Benjamin Hitz', 'Hitz', 'Cannery', 'Mother', 'Lucky Pierre', 'Leora Duncan', 'Duncan', 'Duncan', 'Duncan', 'Leora Duncan', 'Hitz', 'Hitz', 'Zeus', 'Leora Duncan', 'Hitz', 'Duncan', 'Duncan', 'Hitz', 'Leora Duncan', 'Hitz', 'Edward K. Wehling', 'Wehling', 'Hitz', 'Hitz', 'Hitz', 'Hitz', 'Hitz', 'Wehling', 'Hitz', 'Hitz', 'Hitz', 'Hitz', 'Leora Duncan', 'Hitz', 'Hitz', 'Leora Duncan', 'Wehling', 'Hitz', 'Hitz', 'Leora Duncan']


Hitz                 20
Leora Duncan          7
Duncan                5
Wehling               3
Edward K. Wehling     2
Young Wehling         1
Benjamin Hitz         1
Cannery               1
Mother                1
Lucky Pierre          1
Zeus                  1
dtype: int64

In [ ]:
# Tokenize the whole 2BR02B book into a list of word tokens
to_be_or_not_sents = word_tokenizer(list(df["2_B_R_0_2_B"]))


character_dict = {
                  "ed": ["edward k. wehling jr.", "wehling", "edward k. wehling", 'young wehling', "edward", "mr wehling"],
                  "painter": ["the painter", "painter"],
                  "dr": ["dr. ritz", "dr. benjamin hitz", "hitz", "benjamin"],
                  "leora": ["leora duncan", "leora", "duncan", "miss duncan", "ms duncan"],
                  }

# Count frequency of characters
character_freq = count_characters(to_be_or_not_tokens, character_dict)

# Convert to dataframe for easier comparison
name_count_df = pd.DataFrame.from_dict(character_freq, orient='index', columns=["count"])
name_count_df = name_count_df.reset_index()
name_count_df.columns= ["name", "count"]
name_count_df.sort_values("count",ascending=False)

,name,count
0,ed,24
2,dr,20
3,leora,17
1,painter,14


In [ ]:
ed_dialog = flatten_list(get_character_dialogue(to_be_or_not_sents, character_dict, "ed"))
dr_dialog = flatten_list(get_character_dialogue(to_be_or_not_sents, character_dict, "dr"))
leora_dialog = flatten_list(get_character_dialogue(to_be_or_not_sents, character_dict, "leora"))

In [ ]:
ed_dialog

['one bright morning in the chicago lying-in hospital, a man named edward k. wehling, jr. ,  waited for his wife to give birth.',
 'wehling was fifty-six, a mere stripling in a population whose average age was one hundred and twenty-nine.',
 'young wehling was hunched in his chair, his head in his hand.',
 'wehling, the waiting father, mumbled something without raising his head.',
 '"wehling, " said the waiting father, sitting up, red-eyed and frowzy.',
 '"edward k. wehling, jr. , is the name of the happy father-to-be. "',
 '"oh, mr. wehling, " said dr. hitz, "i didn not see you. "',
 '"the invisible man, " said wehling.',
 '"hooray, " said wehling emptily.',
 'said wehling.',
 'dr. hitz became rather severe with wehling, towered over him.',
 'wehling?"',
 '"i think it\'s perfectly keen, " said wehling tautly.',
 'wehling?"',
 '"nope, " said wehling sulkily.',
 '"a drupelet, mr. wehling, is one of the little knobs, one of the little pulpy grains of a blackberry, " said dr. hitz.',
 'we

In [ ]:
leora_dialog

['"my name\'s leora duncan. "',
 '"duncan, duncan, duncan, " he said, scanning the list.',
 '"well, " said leora duncan, "that\'s more the disposal people, isn not it?',
 'and, while leora duncan was posing for her portrait, into the waitingroom bounded dr. hitz himself.',
 '"well, miss duncan!',
 'miss duncan!"',
 'said leora duncan.',
 '"i wish people wouldn not call it that, " said leora duncan.',
 '"that sounds so much better, " said leora duncan.',
 'and then he shot leora duncan.']

## Sentiment Analysis

### Whole collection

In [663]:
# Perform sentiment analysis using BERT
whole_collect = list(df_ori["all"])
result_all_data = sentiment_analyzer(whole_collect)

# Get the classified sentiment                         
positive_num, negative_num = count_polarity(result_all_data)
print('Number of postive labels vs negative labels: ', positive_num, " vs ", negative_num)
print('Ratios of postive labels vs negative labels in the whole collection: ', polarity_ratio(result_all_data, positive_num, negative_num))

loading configuration file https://huggingface.co/siebert/sentiment-roberta-large-english/resolve/main/config.json from cache at /home/evelyn/.cache/huggingface/transformers/228e83e1ade2247aebc5f0725e330fa58dedee3d9eec36c9249f25084a946130.1aece0680a18a95d51d6e1a5f83631412da37b87db65380c52052161354505ba
Model config RobertaConfig {
  "_name_or_path": "siebert/sentiment-roberta-large-english",
  "architectures": [
    "RobertaForSequenceClassification"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "classifier_dropout": null,
  "eos_token_id": 2,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 1024,
  "id2label": {
    "0": "NEGATIVE",
    "1": "POSITIVE"
  },
  "initializer_range": 0.02,
  "intermediate_size": 4096,
  "label2id": {
    "NEGATIVE": 0,
    "POSITIVE": 1
  },
  "layer_norm_eps": 1e-05,
  "max_position_embeddings": 514,
  "model_type": "roberta",
  "num_attention_heads": 16,
  "num_hidden_layers":

Number of postive labels vs negative labels:  14433  vs  45467
Ratios of postive labels vs negative labels in the whole collection:  (0.24095158597662772, 0.7590484140233723)


### for 2BR02B only

In [653]:
# Perform sentiment analysis using BERT
result_to_be_book = sentiment_analyzer(book_sents[1])

# Get the classified sentiment     
to_be_positive_num, to_be_negative_num = count_polarity(result_to_be_book)
print('Number of postive labels vs negative labels: ', to_be_positive_num, " vs ", to_be_negative_num)
print('Ratios of postive labels vs negative labels in the whole collection: ', polarity_ratio(result_to_be_book, to_be_positive_num, to_be_negative_num))

Number of postive labels vs negative labels:  115  vs  144
Ratios of postive labels vs negative labels in the whole collection:  (0.444015444015444, 0.555984555984556)


### for 2BR02B characters

#### Ed

In [655]:
# Perform sentiment analysis using BERT
result_to_be_ed = sentiment_analyzer(ed_dialog)

# Get the classified sentiment     
ed_positive_num, ed_negative_num = count_polarity(result_to_be_ed)
print('Number of postive labels vs negative labels: ', ed_positive_num, " vs ", ed_negative_num)
print('Ratios of postive labels vs negative labels in the whole collection: ', polarity_ratio(result_to_be_ed, ed_positive_num, ed_negative_num))

loading configuration file https://huggingface.co/siebert/sentiment-roberta-large-english/resolve/main/config.json from cache at /home/evelyn/.cache/huggingface/transformers/228e83e1ade2247aebc5f0725e330fa58dedee3d9eec36c9249f25084a946130.1aece0680a18a95d51d6e1a5f83631412da37b87db65380c52052161354505ba
Model config RobertaConfig {
  "_name_or_path": "siebert/sentiment-roberta-large-english",
  "architectures": [
    "RobertaForSequenceClassification"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "classifier_dropout": null,
  "eos_token_id": 2,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 1024,
  "id2label": {
    "0": "NEGATIVE",
    "1": "POSITIVE"
  },
  "initializer_range": 0.02,
  "intermediate_size": 4096,
  "label2id": {
    "NEGATIVE": 0,
    "POSITIVE": 1
  },
  "layer_norm_eps": 1e-05,
  "max_position_embeddings": 514,
  "model_type": "roberta",
  "num_attention_heads": 16,
  "num_hidden_layers":

Number of postive labels vs negative labels:  7  vs  16
Ratios of postive labels vs negative labels in the whole collection:  (0.30434782608695654, 0.6956521739130435)


In [656]:
result_to_be_ed

,text,pred,label,score
0,one bright morning in the chicago lying-in hos...,1,POSITIVE,0.996789
1,"wehling was fifty-six, a mere stripling in a p...",0,NEGATIVE,0.996718
2,"young wehling was hunched in his chair, his he...",0,NEGATIVE,0.998107
3,"wehling, the waiting father, mumbled something...",0,NEGATIVE,0.998563
4,"""wehling, "" said the waiting father, sitting u...",0,NEGATIVE,0.996208
5,"""edward k. wehling, jr. , is the name of the h...",1,POSITIVE,0.998001
6,"""oh, mr. wehling, "" said dr. hitz, ""i didn not...",0,NEGATIVE,0.998021
7,"""the invisible man, "" said wehling.",0,NEGATIVE,0.989193
8,"""hooray, "" said wehling emptily.",0,NEGATIVE,0.998388
9,said wehling.,1,POSITIVE,0.978962


#### Dr

In [673]:
# Perform sentiment analysis using BERT
result_to_be_dr = sentiment_analyzer(dr_dialog)

dr_positive_num, dr_negative_num = count_polarity(result_to_be_dr)
print('Number of postive labels vs negative labels: ', dr_positive_num, " vs ", dr_negative_num)
print('Ratios of postive labels vs negative labels in the whole collection: ', polarity_ratio(result_to_be_dr, dr_positive_num, dr_negative_num))

loading configuration file https://huggingface.co/siebert/sentiment-roberta-large-english/resolve/main/config.json from cache at /home/evelyn/.cache/huggingface/transformers/228e83e1ade2247aebc5f0725e330fa58dedee3d9eec36c9249f25084a946130.1aece0680a18a95d51d6e1a5f83631412da37b87db65380c52052161354505ba
Model config RobertaConfig {
  "_name_or_path": "siebert/sentiment-roberta-large-english",
  "architectures": [
    "RobertaForSequenceClassification"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "classifier_dropout": null,
  "eos_token_id": 2,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 1024,
  "id2label": {
    "0": "NEGATIVE",
    "1": "POSITIVE"
  },
  "initializer_range": 0.02,
  "intermediate_size": 4096,
  "label2id": {
    "NEGATIVE": 0,
    "POSITIVE": 1
  },
  "layer_norm_eps": 1e-05,
  "max_position_embeddings": 514,
  "model_type": "roberta",
  "num_attention_heads": 16,
  "num_hidden_layers":

Number of postive labels vs negative labels:  14  vs  7
Ratios of postive labels vs negative labels in the whole collection:  (0.6666666666666666, 0.3333333333333333)


#### Leora

In [675]:
# Perform sentiment analysis using BERT
result_to_be_leora = sentiment_analyzer(leora_dialog)

leora_positive_num, leora_negative_num = count_polarity(result_to_be_leora)
print('Number of postive labels vs negative labels: ', leora_positive_num, " vs ", leora_negative_num)
print('Ratios of postive labels vs negative labels in the whole collection: ', polarity_ratio(result_to_be_leora, leora_positive_num, leora_negative_num))

loading configuration file https://huggingface.co/siebert/sentiment-roberta-large-english/resolve/main/config.json from cache at /home/evelyn/.cache/huggingface/transformers/228e83e1ade2247aebc5f0725e330fa58dedee3d9eec36c9249f25084a946130.1aece0680a18a95d51d6e1a5f83631412da37b87db65380c52052161354505ba
Model config RobertaConfig {
  "_name_or_path": "siebert/sentiment-roberta-large-english",
  "architectures": [
    "RobertaForSequenceClassification"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "classifier_dropout": null,
  "eos_token_id": 2,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 1024,
  "id2label": {
    "0": "NEGATIVE",
    "1": "POSITIVE"
  },
  "initializer_range": 0.02,
  "intermediate_size": 4096,
  "label2id": {
    "NEGATIVE": 0,
    "POSITIVE": 1
  },
  "layer_norm_eps": 1e-05,
  "max_position_embeddings": 514,
  "model_type": "roberta",
  "num_attention_heads": 16,
  "num_hidden_layers":

Number of postive labels vs negative labels:  4  vs  6
Ratios of postive labels vs negative labels in the whole collection:  (0.4, 0.6)


## Emotion Classification

### Whole collection

In [481]:
collection_sents = flatten_list(book_sents)
# Perform classification
collection_emo = emo_count(collection_sents)
collection_emo

defaultdict(int,
            {'anticipation': 8911,
             'joy': 7125,
             'positive': 18018,
             'surprise': 4944,
             'trust': 10674,
             'negative': 15508,
             'anger': 6239,
             'fear': 8706,
             'sadness': 7923,
             'disgust': 3803})

In [644]:
# Visualize the result of emotion classification
all_emo_df = dict_to_df(collection_emo)
print(visualize_df(all_emo_df, x='Emotion Count', y='Emotion Classification', title="Emotional distribution in the entire collection"))
print(visualize_df(multi_2_binary_df(all_emo_df, multiclass_to_binary_count(all_emo_df, 0)), x="Count", y= "index", title="Binary version of emotional distribution in the entire collection"))

None


None


### for 2BR02B only

In [668]:
to_be_emo = emo_count(book_sents[1])
to_be_df = dict_to_df(to_be_emo)
print(visualize_df(to_be_df, x='Emotion Count', y='Emotion Classification', title="Emotional distribution in the 2BR02B book"))
print(visualize_df(multi_2_binary_df(to_be_df, multiclass_to_binary_count(to_be_df, 0)), x="Count", y= "index", title="Binary version of emotional distribution in the 2BR02B book"))

None


None


### for 2BR02B characters

In [636]:
#Character emotion profiling

ed_emo = emo_count(ed_dialog)
dr_emo = emo_count(dr_dialog)
leora_emo = emo_count(leora_dialog)
print("Edward's emotion distribution is", sort_dict(ed_emo))
print("Dr's emotion distribution is", sort_dict(dr_emo))
print("Leora's emotion distribution is", sort_dict(leora_emo))

Edward's emotion distribution is [('trust', 8), ('positive', 7), ('fear', 6), ('joy', 5), ('negative', 5), ('sadness', 4), ('anticipation', 4), ('surprise', 3), ('anger', 3)]
Dr's emotion distribution is [('positive', 10), ('trust', 9), ('joy', 6), ('anticipation', 5), ('sadness', 3), ('negative', 3), ('surprise', 2), ('fear', 2), ('disgust', 1), ('anger', 1)]
Leora's emotion distribution is [('negative', 2), ('anger', 1), ('fear', 1), ('sadness', 1), ('surprise', 1)]


#### Ed

In [875]:
ed_emo_df = dict_to_df(ed_emo)
print(ed_emo_df)
print(visualize_df(ed_emo_df, x='Emotion Count', y='Emotion Classification', title="Edward Emotional Distribution")) #27/45 = 0.6
print(visualize_df(multi_2_binary_df(ed_emo_df, multiclass_to_binary_count(ed_emo_df, 0)), x="Count", y= "index", title="Binary version of Edward Emotional Distribution"))

  Emotion Classification  Emotion Count
2                  trust              8
5               positive              7
0                   fear              6
4                    joy              5
7               negative              5
1                sadness              4
3           anticipation              4
6               surprise              3
8                  anger              3


None


None


#### Dr

In [876]:
dr_emo_df = dict_to_df(dr_emo)
print(dr_emo_df)
print(visualize_df(dr_emo_df, x='Emotion Count', y='Emotion Classification', title="Dr Emotional Distribution")) #27/45 = 0.6
print(visualize_df(multi_2_binary_df(dr_emo_df, multiclass_to_binary_count(dr_emo_df, 0)), x="Count", y= "index", title="Binary version of Dr Emotional Distribution"))

  Emotion Classification  Emotion Count
2               positive             10
4                  trust              9
1                    joy              6
0           anticipation              5
6                sadness              3
8               negative              3
3               surprise              2
5                   fear              2
7                disgust              1
9                  anger              1


None


None


#### Leora

In [880]:
leora_emo_df = dict_to_df(leora_emo)
print(leora_emo_df)
print(visualize_df(leora_emo_df, x='Emotion Count', y='Emotion Classification', title="Leora's Emotional Distribution")) #27/45 = 0.6
print(visualize_df(multi_2_binary_df(leora_emo_df, multiclass_to_binary_count(leora_emo_df, 0)), x="Count", y= "index", title="Binary version of Leora's Emotional Distribution"))

  Emotion Classification  Emotion Count
0               negative              2
1                  anger              1
2                   fear              1
3                sadness              1
4               surprise              1


None


None


## Evaluation

Considering the lack of labeled data and time constraints, I only manually labeled the dataset of the 3 characters in 2BR02B to create an evaluation set. Therefore, automatic evaluation is only performed for the chatacters of the subset of the data. The results from the evaluation set will be a represenation to the entire dataset. The rest of the results is manually evaluated.

In [ ]:
# Load eval data

eval_bi_leora = pd.read_csv("/home/evelyn/Documents/digiphil/final_project/data_label/labeled_leora.csv", converters={'actual_emo': lambda x: x[:].split(',')})
eval_bi_dr = pd.read_csv("/home/evelyn/Documents/digiphil/final_project/data_label/labeled_dr.csv", converters={'actual_emo': lambda x: x[:].split(',')})
eval_bi_ed= pd.read_csv("/home/evelyn/Documents/digiphil/final_project/data_label/labeled_ed.csv", converters={'actual_emo': lambda x: x[:].split(',')})

In [ ]:
eval_bi_leora

,text,actual_label,pred,label,actual_emo
0,"""my name's leora duncan. """,1,1,POSITIVE,[positive]
1,"""duncan, duncan, duncan, "" he said, scanning t...",0,0,NEGATIVE,[anticipation]
2,"""well, "" said leora duncan, ""that's more the d...",0,0,NEGATIVE,[negative]
3,"and, while leora duncan was posing for her por...",1,1,POSITIVE,"[positive, surprise]"
4,"""well, miss duncan!",1,0,NEGATIVE,"[positive, surprise]"
5,"miss duncan!""",1,0,NEGATIVE,[positive]
6,said leora duncan.,1,1,POSITIVE,[neutral]
7,"""i wish people wouldn not call it that, "" said...",0,0,NEGATIVE,[sadness]
8,"""that sounds so much better, "" said leora duncan.",1,1,POSITIVE,[positive]
9,and then he shot leora duncan.,0,0,NEGATIVE,"[sadness, anger, negative]"


### Sentiment analysis

In [ ]:
# accuracy score
print("Leora accuracy score: ", compute_accuracy(list(eval_bi_leora["actual_label"]), list(result_to_be_leora['pred'])))
print("Dr accuracy score: ",compute_accuracy(list(eval_bi_dr["actual_label"]), list(result_to_be_dr['pred'])))
print("Ed accuracy score: ",compute_accuracy(list(eval_bi_ed["actual_label"]), list(result_to_be_ed['pred'])))

Leora accuracy score:  0.8
Dr accuracy score:  0.8571428571428571
Ed accuracy score:  0.8260869565217391


In [ ]:
# macro F1 score 
print("Leora F1 score: ", f1_score(list(eval_bi_leora["actual_label"]), list(result_to_be_leora['pred']), average='macro'))
print("Dr F1 score: ", f1_score(list(eval_bi_dr["actual_label"]), list(result_to_be_dr['pred']), average='macro'))
print("Ed F1 score: ",f1_score(list(eval_bi_ed["actual_label"]), list(result_to_be_ed['pred']), average='macro'))

Leora F1 score:  0.8
Dr F1 score:  0.8444444444444444
Ed F1 score:  0.8083333333333333


In [ ]:
# micro F1 score
print("Leora F1 score: ", f1_score(list(eval_bi_leora["actual_label"]), list(result_to_be_leora['pred']), average='micro'))
print("Dr F1 score: ", f1_score(list(eval_bi_dr["actual_label"]), list(result_to_be_dr['pred']), average='micro'))
print("Ed F1 score: ",f1_score(list(eval_bi_ed["actual_label"]), list(result_to_be_ed['pred']), average='micro'))

Leora F1 score:  0.8000000000000002
Dr F1 score:  0.8571428571428571
Ed F1 score:  0.8260869565217391


### Emotion classification 

In [851]:
# See the predicted emotion for each characters, 
# assign NaN if predicted class is empty

leora_emo_pred_df = emo_df_for_eval(leora_dialog)
dr_emo_pred_df = emo_df_for_eval(dr_dialog)
ed_emo_pred_df = emo_df_for_eval(ed_dialog)
leora_emo_pred_df # check

,sentence,emo_pred
0,"""my name's leora duncan. """,[NaN]
1,"""duncan, duncan, duncan, "" he said, scanning t...",[NaN]
2,"""well, "" said leora duncan, ""that's more the d...",[negative]
3,"and, while leora duncan was posing for her por...",[NaN]
4,"""well, miss duncan!",[NaN]
5,"miss duncan!""",[NaN]
6,said leora duncan.,[NaN]
7,"""i wish people wouldn not call it that, "" said...",[NaN]
8,"""that sounds so much better, "" said leora duncan.",[NaN]
9,and then he shot leora duncan.,"[anger, fear, negative, sadness, surprise]"


### Evaluating the emotion classification 


In [ ]:
# Clean and normalize the dataframes for evaluation
for_emo_eval_leora = clean_df_for_emoeval(eval_bi_leora)
for_emo_eval_dr = clean_df_for_emoeval(eval_bi_dr)
for_emo_eval_ed = clean_df_for_emoeval(eval_bi_ed)

normalized_emo_pred_df_leora = normalize_NA_pred(for_emo_eval_leora, leora_emo_pred_df)
normalized_emo_pred_df_dr = normalize_NA_pred(for_emo_eval_dr, dr_emo_pred_df)
normalized_emo_pred_df_ed = normalize_NA_pred(for_emo_eval_ed, ed_emo_pred_df)

In [873]:
# Accuracy score
print("Leora accuracy score: ", compute_accruacy_multilabel(for_emo_eval_leora, normalized_emo_pred_df_leora))
print("Dr accuracy score: ", compute_accruacy_multilabel(for_emo_eval_dr, normalized_emo_pred_df_dr))
print("Ed accuracy score: ", compute_accruacy_multilabel(for_emo_eval_ed, normalized_emo_pred_df_ed))

Leora accuracy score:  0.13333333333333333
Dr accuracy score:  0.39682539682539686
Ed accuracy score:  0.25362318840579706
